__Описание__

В данном примере рассматривается двумерное вейвлет преобразование. Рассмтривается процесс изменения спектральной области изображения. Также рассматривается синтез изображения из двух исходных.

Данный пример может быть загружен по адресу: https://github.com/sven4500/num-analysis

In [ ]:
import numpy as np
import pywt
import matplotlib
from matplotlib import pyplot
from matplotlib import image

Работать с цветным изображением сложнее поэтому определяем функцию преобразования цветного изображения в чёрно-белое.

In [ ]:
def to_grayscale(img):
    #return np.dot(img[..., :3], [0.2989, 0.5870, 0.1140])
    return np.dot(img[..., :3], [0.3333, 0.3333, 0.3333])

In [ ]:
def side_by_side(img_1, img_2, title_1='Оригинал', title_2='Обработанный'):
    fig, ax = matplotlib.pyplot.subplots(1, 2)
    fig.set_dpi(180)

    ax[0].set_title(title_1)
    ax[0].imshow(img_1, cmap='gray')

    ax[1].set_title(title_2)
    ax[1].imshow(img_2, cmap='gray')

Считываем изображение из файла.

In [ ]:
path = '.\photo.png'

img = matplotlib.image.imread(path)
img = to_grayscale(img)

fig = matplotlib.pyplot.figure(dpi=180)
#ax = fig.add_axes()
matplotlib.pyplot.imshow(img, cmap='gray', interpolation='none')

Для двумерного изображения функция __wavedec2__ возвращает группу разложенных изображений (__анализ__) в следующем порядке: [A] [Hn, Vn, Dn] ... [H1, V1, D1], где H, V, D горизонтальные, вертикальные и даиагональные особенности сигнала, а n соответствует номеру уровня. Таким образом каждая группа соответствует своему уровню преобразования.

В примере ниже горизонтальные компонента сигнала (1) задаётся как константное значение. Как результат видим картинку в которой имеются вертикальные полосы. Сигнал аппроксимаций A не трогаем.

In [ ]:
wavelet = 'sym20' # <- симмлет (англ. symmetry wavelet) модифицированная версия вейвлета Добеши с повышенной симметрией
level = 3 # <- количество уровней дискретного разложения

dwt = pywt.wavedec2(img, wavelet=wavelet, level=level)

for d in dwt[1:]:
    m, n = d[1].shape
    mean = np.sqrt(np.sum(np.square(d[1])) / (m * n))
    d[1][:] = 2. * mean

Далее следует обратное вейвлет разложение (__синтез__).

In [ ]:
idwt = pywt.waverec2(dwt, wavelet=wavelet)
side_by_side(img, idwt)

В следующем примере рассмотрено слияние двух изображений.

In [ ]:
path_2 = '.\photo_2.png'
img_2 = matplotlib.image.imread(path_2)
img_2 = to_grayscale(img_2)

side_by_side(img, img_2, 'Изображение 1', 'Изображение 2')

Поперечные и продольные компоненты обоих изображений не обязательно должны смешиваться пропорционально. В примере ниже они смешиваются в пропорциях 0.7 к 0.3. В этом также необходимо смешивать коэффициенты аппроксимации.

In [ ]:
wvaelet = 'sym20'
level = 3

dwt_1 = pywt.wavedec2(img, wavelet=wavelet, level=level)
dwt_2 = pywt.wavedec2(img_2, wavelet=wavelet, level=level)

dwt_1[0] = 0.7 * dwt_1[0] + 0.3 * dwt_2[0]

for d_1, d_2 in zip(dwt_1[1:], dwt_2[1:]):
    d_1[0][:] = 0.7 * d_1[0] + 0.3 * d_2[0]
    d_1[1][:] = 0.7 * d_1[1] + 0.3 * d_2[1]
    d_1[2][:] = 0.7 * d_1[2] + 0.3 * d_2[2]

Далее производим __синтез__ результирующего изображения.

In [ ]:
idwt = pywt.waverec2(dwt_1, wavelet=wavelet)

side_by_side(img, idwt, 'Оригинал', 'Объединение')

__Итоги__

Был рассмотрен мультиуровневый способ вейвлет разложения изображений. Было рассмотрен вариент слияния двух изображений при помощи вейвлет преобразования.

__Источники__

[1] 2D Forward and Inverse Discrete Wavelet Transform // https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html

[2] Introduction to Wavelets in Image
Processing // http://inside.mines.edu/~whoff/courses/EENG510/lectures/